In [32]:
import pandas as pd
import tqdm
from tqdm import tqdm
import numpy as np
import scipy.sparse as sparse
import random
from implicit.als import AlternatingLeastSquares
import os
from fastparquet import write
import pickle

In [3]:
url = '../dataset/'
df = pd.read_parquet(url+'dataset.parquet')

In [4]:
df.head()

,date,time,user_id,event_type,product_id,category_id,first_level,second_level,third_level,brand,price,user_session
0,2019-10-01,00:00:00,554748717,view,3900821,2053013552326770905,appliances,environment,water_heater,aqua,33.20,57-6336556b0fc
1,2019-10-01,00:00:05,512742880,view,1480613,2053013561092866779,computers,desktop,desktop,pulser,908.62,a5-86594dec0db
2,2019-10-01,00:00:08,550978835,view,31500053,2053013558031024687,empty,empty,empty,luminarc,41.16,a7-abc6048498d
3,2019-10-01,00:00:11,555158050,view,2900536,2053013554776244595,appliances,kitchen,microwave,elenberg,51.46,9e-9ce44bb50ab
4,2019-10-01,00:00:22,512742880,view,1480714,2053013561092866779,computers,desktop,desktop,pulser,921.49,a5-86594dec0db


In [5]:
# 아이템 설명 테이블
item_lookup = df[['product_id','category_id','first_level','second_level','third_level','brand','price']]

In [6]:
item_lookup.drop_duplicates(subset='product_id', inplace=True)

/var/folders/g9/_25j58_92h369x7v6drzc3g80000gn/T/ipykernel_41769/412940082.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  item_lookup.drop_duplicates(subset='product_id', inplace=True)


In [7]:
item_lookup.product_id.value_counts()

3900821     1
7201705     1
18900404    1
45600530    1
12705282    1
           ..
12300815    1
12715498    1
3300410     1
14700321    1
4201348     1
Name: product_id, Length: 146872, dtype: int64

In [8]:
user_log = df[['user_id','product_id','event_type']]

In [9]:
user_log.head()

,user_id,product_id,event_type
0,554748717,3900821,view
1,512742880,1480613,view
2,550978835,31500053,view
3,555158050,2900536,view
4,512742880,1480714,view


In [10]:
grouped_df = user_log.groupby(['user_id','product_id','event_type']).size().to_frame().reset_index()

In [11]:
grouped_df = grouped_df[grouped_df['event_type'] == 'purchase']

In [12]:
grouped_df.head()

,user_id,product_id,event_type,0
349,340041246,2100099,purchase,1
361,340041246,9300037,purchase,3
796,384989212,2501816,purchase,1
932,387300134,10700971,purchase,1
1499,410333972,1003304,purchase,1


In [13]:
pivot_table = grouped_df.pivot(index='user_id',columns='product_id',values=0)

/var/folders/g9/_25j58_92h369x7v6drzc3g80000gn/T/ipykernel_41769/1928844478.py:1: PerformanceWarning: The following operation may generate 2845991526 cells in the resulting pandas object.
  pivot_table = grouped_df.pivot(index='user_id',columns='product_id',values=0)


In [14]:
pivot_table.fillna(0, inplace=True)

In [15]:
pivot_table.values

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [16]:
# rows = []
# cols = []
# values = []
# for row in tqdm(pivot_table.index):
#     for col in pivot_table.columns:
#         if pivot_table.loc[row,col] == 0.0:
#             continue
#         else:
#             # print(pivot_table.loc[row,col])
#             rows.append(row)
#             cols.append(col)
#             values.append(int(pivot_table.loc[row,col]))


100%|██████████| 115213/115213 [4:39:30<00:00,  6.87it/s] 


In [24]:
# rows = []
# cols = []
# values = []
# for row in tqdm(range(len(pivot_table[:100].index))):
#     for col in range(len(pivot_table[:100].columns)):
#         if pivot_table.iloc[row,col] == 0.0:
#             continue
#         else:
#             # print(pivot_table.loc[row,col])
#             rows.append(row)
#             cols.append(col)
#             values.append(int(pivot_table.iloc[row,col]))

100%|██████████| 100/100 [00:29<00:00,  3.42it/s]


In [16]:
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

In [17]:
pivot_table

product_id,1000978,1001588,1002062,1002098,1002099,1002100,1002101,1002102,1002225,1002266,...,57100091,57100098,57800001,57800002,57800005,58000000,58300001,58300012,60400006,60500001
user_id,,,,,,,,,,,,,,,,,,,,,
340041246,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
384989212,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
387300134,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
410333972,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
413782099,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
566258816,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
566267125,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
566270060,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [18]:
user_idx = {i:j for j,i in enumerate(pivot_table.index)}
product_idx = {i:j for j,i in enumerate(pivot_table)}

In [19]:
input_data = pivot_table.T

In [20]:
tocsrdata = sparse.csr_matrix(pivot_table.values)

In [21]:
matrix_size = tocsrdata.shape[0]* tocsrdata.shape[1]
num_purchases = len(tocsrdata.nonzero()[0])
sparsity = 100 * (1 - (num_purchases / matrix_size))
sparsity

99.99351343817037

In [22]:
model = AlternatingLeastSquares(factors=20)

In [24]:
model.fit(tocsrdata)

100%|██████████| 15/15 [01:47<00:00,  7.18s/it]


In [34]:
with open('model.pickle','wb') as fw:
    pickle.dump(model,fw)

In [35]:
product_idx[1000978]

0

In [25]:
model.similar_items(product_idx[1000978])

(array([   0, 1375, 1374, 1373, 9928, 7032, 1426, 1408,    7, 6054],
       dtype=int32),
 array([1.        , 0.98351485, 0.9835046 , 0.9834741 , 0.9833915 ,
        0.98234814, 0.96949226, 0.9557605 , 0.9547173 , 0.92450523],
       dtype=float32))

In [26]:
recc_item_list = model.similar_items(product_idx[1000978])

In [27]:
idx_product = {j:i for i,j in product_idx.items()}

In [28]:
recc_item_list[0]

array([   0, 1375, 1374, 1373, 9928, 7032, 1426, 1408,    7, 6054],
      dtype=int32)

In [29]:
item_lookup[item_lookup['product_id'] == 1000978]

,product_id,category_id,first_level,second_level,third_level,brand,price
9297792,1000978,2053013555631882655,electronics,smartphone,smartphone,samsung,0.0


In [67]:
for i in recc_item_list[0]:
    item_id = idx_product[i]
    display(item_lookup[item_lookup['product_id'] == item_id])

,product_id,category_id,first_level,second_level,third_level,brand,price
9297792,1000978,2053013555631882655,electronics,smartphone,smartphone,samsung,0.0


,product_id,category_id,first_level,second_level,third_level,brand,price
37678,1480196,2053013561092866779,computers,desktop,desktop,lenovo,463.07


,product_id,category_id,first_level,second_level,third_level,brand,price
352488,1480195,2053013561092866779,computers,desktop,desktop,lenovo,720.71


,product_id,category_id,first_level,second_level,third_level,brand,price
255360,1480194,2053013561092866779,computers,desktop,desktop,lenovo,385.85


,product_id,category_id,first_level,second_level,third_level,brand,price
292987,11700098,2053013554591695207,electronics,audio,acoustic,samson,939.54


,product_id,category_id,first_level,second_level,third_level,brand,price
3514091,6301412,2053013554834964853,appliances,kitchen,kettle,willmark,41.16


,product_id,category_id,first_level,second_level,third_level,brand,price
281576,1480600,2053013561092866779,computers,desktop,desktop,acer,587.66


,product_id,category_id,first_level,second_level,third_level,brand,price
56978,1480466,2053013561092866779,computers,desktop,desktop,hp,503.54


,product_id,category_id,first_level,second_level,third_level,brand,price
11189,1002102,2053013555631882655,electronics,smartphone,smartphone,samsung,414.53


,product_id,category_id,first_level,second_level,third_level,brand,price
58939,3300315,2053013555355058573,empty,empty,empty,redmond,113.23
